# Qutrit Machine Learning Optimizaiton 

Author: Bora Basyildiz

## Imports

In [1]:
import torch
import numpy as np
from itertools import permutations
from itertools import product
import matplotlib.pyplot as plt

## Qutrit Fidelity Calculation

In [2]:
def fidelity_Qutrit(J,B,M,input_gate,t,N_iter):
    #!/usr/bin/env python3
    # -*- coding: utf-8 -*-
    """
    Created on Mon Aug 16 4:33 2021

    @author: Bora & Alex
    """
    #imports

    #Pauli Matricies 
    l1 = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]) 
    l2 = np.array([[0,-1j, 0],[1j,0, 0], [0, 0, 0]]) 
    l3 = np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]]) 
    l4 = np.array([[0,0,1],[0,0,0],[1,0,0]])
    l5 = np.array([[0,0,-1j],[0,0,0],[1j,0,0]])
    l6 = np.array([[0,0,0],[0,0,1],[0,1,0]]) #essentially X for 1->2 transition
    l7 = np.array([[0,0,0],[0,0,-1j],[0,1j,0]]) #essentially Y for 1->2 transition
    l8 = 1/np.sqrt(3) * np.array([[1,0,0],[0,1,0],[0,0,-2]])
    id = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])

    sx = np.array([[0,1,0],[1,0,0],[0,0,1]])
    sy = np.array([[0,-1j,0],[1j,0,0],[0,0,1]])
    sx2 = np.array([[1,0,0],[0,0,1],[0,1,0]])
    sy2 = np.array([[1,0,0],[0,0,-1j],[0,1j,0]])
    sx02 = np.array([[0,0,1],[0,1,0],[1,0,0]])
    sy02 = np.array([[0,0,-1j],[0,1,0],[1j,0,0]])
    sz = np.array([[1,0,0],[0,-1,0],[0,0,1]])
    
    #Function definitions 
    def zero_mat(N):#Generates matrix of zeros
        zero_gate = np.array([[0,0,0],[0,0,0],[0,0,0]])
        init = zero_gate
        if N < 2:
            return 1
        for i in range(0,N - 1):
            zero_gate = torch.tensor(np.kron(zero_gate,init),dtype=torch.cdouble)
        return zero_gate
    def sum_pauli(coef, gate):#Sums Pauli gates with coefficients 
        N = len(coef)#number of qubits
        total_pauli = zero_mat(N)
        #Summing all Z gates
        for i in range(0,N):
            pauli_temp = 1
            for j in range(0,i):
                pauli_temp = torch.tensor(np.kron(pauli_temp,id))
            pauli_temp = torch.tensor(np.kron(pauli_temp,gate))
            for j in range(i+1,N):
                pauli_temp = torch.tensor(np.kron(pauli_temp,id))
            total_pauli = total_pauli + coef[i]*pauli_temp
        #return torch.tensor(total_pauli,dtype=torch.cdouble)
        return total_pauli

    #variable initializations
    N = len(B)
    torch.manual_seed(1)
    dt = torch.cdouble # datatype and precision
    infidelity_list=torch.zeros([N_iter,1])

    #J coefficients gathering (only if J is in N x N matrix, otherwise set J_coef=J) <- essentially flattens the array
    J_coef = []
    for i in range(0,len(J) - 1):
        for j in range(0,len(J) - i - 1):
            J_coef.append(J[i,j].item())

    #H0 generation
    permuts = [1,1]
    for i in range(2,N):
        permuts.append(0)
    permuts = list(set(permutations(permuts,N)))
    permuts.sort()
    permuts.reverse()#All permutations of ZZ coupling stored as bit arrays
    H0 = zero_mat(N)
    ##Changed gates ZZ -> XX
    for i,u in enumerate(permuts):#summing ZZ permutations and J constants
        XX_temp = 1
        for p in u:
            if p==1:
                XX_temp = torch.tensor(np.kron(XX_temp,l1))
            else:
                XX_temp = torch.tensor(np.kron(XX_temp,id))
        H0 = H0 + J_coef[i]*XX_temp
    #H0 = H0 + sum_pauli(B,sz)

    #These are the coefficients we are optimizing
    R = torch.rand([M,4*N], dtype=torch.double) *2*np.pi # Random initialization (between 0 and 2pi)
    R.requires_grad = True # set flag so we can backpropagate

    #Optimizer settings(can be changed & opttimized)
    lr=0.3#learning rate

    opt = 'SGD'  # Choose optimizer - ADAM, SGD (typical). ADAMW, ADAMax, Adadelta,  
                        # Adagrad, Rprop, RMSprop, ASGD, also valid options.     
    sched = 'Plateau'  # Choose learning rate scheduler - Plateau, Exponential (typical), Step
    
    if opt=='ADAM': optimizer = torch.optim.Adam([R], lr = lr, weight_decay=1e-6)
    elif opt=='ADAMW': optimizer = torch.optim.AdamW([R], lr = lr, weight_decay=0.01)
    elif opt=='ADAMax': optimizer = torch.optim.Adamax([R], lr = lr, weight_decay=0.01)
    elif opt=='RMSprop': optimizer = torch.optim.RMSprop([R], lr = lr, momentum=0.2)
    elif opt=='Rprop': optimizer = torch.optim.Rprop([R], lr = lr)
    elif opt=='Adadelta': optimizer = torch.optim.Adadelta([R], lr = lr) 
    elif opt=='Adagrad': optimizer = torch.optim.Adagrad([R], lr = lr)
    elif opt=='SGD': optimizer = torch.optim.SGD([R], lr = lr, momentum=0.99, nesterov=True)
    elif opt=='ASGD': optimizer = torch.optim.ASGD([R], lr = lr)
    else: optimizer=None; opt='None'
        
    if sched=='Step': scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=N_iter/10, gamma=0.9)
    elif sched=='Exponential': scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
    elif sched=='Plateau': scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',min_lr=0.03, factor=0.3 , patience= 20 ); loss_in=True; 
    else: scheduler=None; sched='None'

    for n in range(0,N_iter):
        #Creating Drive Hamilontian
        U_Exp = 1
        for i in range(0,N):
            U_Exp = torch.tensor(np.kron(U_Exp,id),dtype=dt)#initializing unitary
        for m in range(0,M):#Product of pulses
            pulse_coef = R[m]
            #H1 = sum_pauli(pulse_coef[:N],l1) + sum_pauli(pulse_coef[N:2*N],l2) + sum_pauli(pulse_coef[2*N:3*N],l4) + sum_pauli(pulse_coef[3*N:4*N],l5) + sum_pauli(pulse_coef[4*N:5*N],l6) + sum_pauli(pulse_coef[5*N:],l7) 
            H1 = sum_pauli(pulse_coef[:N],l1) + sum_pauli(pulse_coef[N:2*N],l2) + sum_pauli(pulse_coef[2*N:3*N],l4) 
            #H1 = sum_pauli(pulse_coef[:N],l1) + sum_pauli(pulse_coef[N:2*N],l2) 
            U_Exp = torch.matmul(torch.matrix_exp(-1j*(H0+H1)*t/M),U_Exp)
        
        #Orthonormal Matrix Generation 
        def Matrix_Basis_Gen(d):
            X = np.zeros([d,d])
            for i in range(d):
                X[(i+1) % d,i] = 1
            Z = np.zeros([d,d],dtype=np.complex_)
            for i in range(d):
                Z[i,i] = np.exp((2*np.pi * 1j * i)/d)
            Basis = []
            for i in range(d):
                for j in range(d):
                    Basis.append(torch.tensor(np.matmul(np.linalg.matrix_power(X,i),np.linalg.matrix_power(Z,j)),dtype=torch.cdouble))
            return Basis

        #Fidelity calulcation given by Nielsen Paper
        fidelity = 0
        d = 3**N
        
        for U in Matrix_Basis_Gen(d):
            ideal_U = torch.matmul(torch.matmul(input_gate,U.conj().T),(input_gate.conj().T))
            target_U = torch.matmul(torch.matmul(U_Exp,U),(U_Exp.conj().T)) # This is Eps(U) = pulse_gate * pauli * pulse_gate^H
            tr = torch.trace(torch.matmul(ideal_U,target_U))
            fidelity = fidelity + tr 
        fidelity = abs(fidelity + d**2)/(d**2 *(d+1))    
        infidelity = 1 - fidelity
        infidelity_list[n] = infidelity.detach()
        infidelity.backward()

        #Printing statement
        if (n+1)%50==0: 
            print('Itertation ', str(n+1), ' out of ', str(N_iter), 'complete. Avg Infidelity: ', str(infidelity.item()))

        #optimizer 
        if optimizer is not None and scheduler is None:  # Update R
            optimizer.step()
            optimizer.zero_grad()
        elif optimizer is not None and scheduler is not None:
            optimizer.step()
            if loss_in: 
                scheduler.step(infidelity)
            else: 
                scheduler.step()
            optimizer.zero_grad()
        else:
            R.data.sub_(lr*R.grad.data) # using data avoids overwriting tensor object
            R.grad.data.zero_()           # and it's respective grad info
    
    #print('The infidelity of the generated gate is: ' + str(infidelity_list.min().item()))
    return R
    #return infidelity_list.min().item()

## Qutrit Testing

In [3]:
CNOT_qutrit = torch.tensor([[1,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0],[0,0,0,1,0,0,0,0,0],[0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,1]],dtype=torch.cdouble)
Identity = torch.tensor([[1,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0],[0,0,0,1,0,0,0,0,0],[0,0,0,0,1,0,0,0,0],[0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,1]],dtype=torch.cdouble)
J = torch.tensor([[1,1],[1,1]])
pulse_frequencies = fidelity_Qutrit(J,[1,1],24,CNOT_qutrit,np.pi,666)

Itertation  50  out of  666 complete. Avg Infidelity:  0.793482530935161
Itertation  100  out of  666 complete. Avg Infidelity:  0.41943588643418594
Itertation  150  out of  666 complete. Avg Infidelity:  0.15253006079235043
Itertation  200  out of  666 complete. Avg Infidelity:  0.0976512709264249
Itertation  250  out of  666 complete. Avg Infidelity:  0.041182241738322634
Itertation  300  out of  666 complete. Avg Infidelity:  0.018795269193259023
Itertation  350  out of  666 complete. Avg Infidelity:  0.014275980275554101
Itertation  400  out of  666 complete. Avg Infidelity:  0.010287948079664488
Itertation  450  out of  666 complete. Avg Infidelity:  0.0064569442890191064
Itertation  500  out of  666 complete. Avg Infidelity:  0.0038551446461007055
Itertation  550  out of  666 complete. Avg Infidelity:  0.0023545154930116885
Itertation  600  out of  666 complete. Avg Infidelity:  0.0020323419737325876
Itertation  650  out of  666 complete. Avg Infidelity:  0.0013171225674685205


In [4]:
pulse_frequencies

tensor([[-2.7287e+00, -2.0758e+00,  6.2595e+00, -8.5197e-01,  4.1646e+00,
         -1.6696e+00,  2.8960e+00,  4.4508e+00],
        [ 6.6742e-01,  2.4075e+00,  3.2326e-01,  2.0354e+00,  9.3165e-01,
         -5.6506e-01,  5.6421e-01,  7.3800e-01],
        [ 5.4242e+00,  6.4310e+00,  5.2013e+00, -3.2477e+00,  1.4526e+00,
          4.7958e+00,  5.6092e+00,  5.5086e+00],
        [ 9.9102e+00, -6.5240e-01,  1.7721e+00,  8.1323e+00,  3.1306e+00,
          1.2981e+00,  2.6033e+00,  2.6918e+00],
        [ 6.0507e+00,  7.7076e-01,  1.0766e+01,  5.0666e+00,  4.9137e+00,
         -6.8607e-01,  2.8551e+00,  1.8570e+00],
        [ 2.4293e+00, -1.3757e+00,  3.5485e+00,  2.2085e+00, -1.2807e+00,
          9.7935e-01,  5.0491e+00,  2.0217e+00],
        [ 2.0788e+00, -5.2428e-01,  8.6794e+00,  4.1052e+00,  1.5588e+00,
          8.8245e+00,  4.6611e+00,  1.8001e+00],
        [ 4.2209e+00,  1.6565e+00,  1.2445e+00,  2.6014e-01,  3.8382e+00,
          7.8819e+00,  8.2639e-01,  1.5025e+00],
        [ 2.1618

## Qutrit Matrix Basis Generation Explanation 

We want a matrix basis that is orthonormal AND unitary. While the Gell-Mann matrices are orthonormal, they are not unitary. Now we will generate these matrices through the method outlined by the Nielsen paper 

In [4]:
def Matrix_Basis_Gen(d):
            X = np.zeros([d,d])
            for i in range(d):
                X[(i+1) % d,i] = 1
            Z = np.zeros([d,d],dtype=np.complex_)
            for i in range(d):
                Z[i,i] = np.exp((2*np.pi * 1j * i)/d)
            Basis = []
            for i in range(d):
                for j in range(d):
                    Basis.append(torch.tensor(np.matmul(np.linalg.matrix_power(X,i),np.linalg.matrix_power(Z,j)),dtype=torch.cdouble))
            return Basis

In [5]:
Matrix_Basis_Gen(2)

[tensor([[1.+0.j, 0.+0.j],
         [0.+0.j, 1.+0.j]], dtype=torch.complex128),
 tensor([[ 1.+0.0000e+00j,  0.+0.0000e+00j],
         [ 0.+0.0000e+00j, -1.+1.2246e-16j]], dtype=torch.complex128),
 tensor([[0.+0.j, 1.+0.j],
         [1.+0.j, 0.+0.j]], dtype=torch.complex128),
 tensor([[ 0.+0.0000e+00j, -1.+1.2246e-16j],
         [ 1.+0.0000e+00j,  0.+0.0000e+00j]], dtype=torch.complex128)]